# SAC Agent Training and Evaluation

## Импорт необходимых модулей

In [1]:
!set PYTHONIOENCODING=utf-8 && pytest --capture=tee-sys

"pytest" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
!set TORCH_USE_CUDA_DSA=1

In [3]:
import os
import sys
import time
import json
import torch
import numpy as np
import gymnasium as gym
import mlflow

In [4]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [5]:
from core.training.trainer import train_agent
from core.evaluation.evaluator import evaluate_agent
from core.sac.factories import EnvFactory

2025-05-12 00:12:53 - sac_app:115 - INFO - Логгер настроен. Консоль: INFO, Файл: INFO


## Тестирование

In [5]:
import os, sys, ipytest
ipytest.autoconfig()

import pytest
pytest.main(["../tests", "-v", "--disable-warnings"])

============================= test session starts =============================
platform win32 -- Python 3.8.16, pytest-8.3.5, pluggy-1.5.0
rootdir: c:\Users\artem\Projects\rl-race-1
configfile: pytest.ini
plugins: anyio-4.5.2, typeguard-4.3.0
collected 29 items

..\tests\test_alpha_component.py .                                       [  3%]
..\tests\test_batching.py ....                                           [ 17%]
..\tests\test_critic_networks.py ....                                    [ 31%]
..\tests\test_device_manager.py ....                                     [ 44%]
..\tests\test_flake8.py s                                                [ 48%]
..\tests\test_losses.py ..                                               [ 55%]
..\tests\test_mypy.py s                                                  [ 58%]
..\tests\test_normalizers.py ..                                          [ 65%]
..\tests\test_policy_networks.py ...                                     [ 75%]
..\tests\test_re

<ExitCode.OK: 0>

## 1. Настройка конфигурации

### Настраиваем MLflow (если нужен удаленный сервер)

mlflow server --host 127.0.0.1 --port 5000 --backend-store-uri file:./mlruns

In [4]:
mlflow.set_tracking_uri("http://localhost:5000")

### Базовая конфигурация для обучения

In [5]:
config = {
    # --- Общие параметры Trainer & Logger ---
    'seed': 42,
    'exp_name': 'sac_lstm_unity_prio_seq', 
    'device': 'cuda',
    'save_dir': 'results',          # Используется TrainingLogger для сохранения config.json и метрик
    'model_name': 'sac_agent',      # Используется SACTrainer для имен чекпоинтов
    'save_checkpoints': True,      # Нужно ли сохранять чекпоинты
    'keep_last_n_checkpoints': 3,  # Количество сохраняемых чекпоинтов (0 - хранить все)
    'load_checkpoint_path': None, # Путь для загрузки чекпоинта при старте
    'log_interval': 1000,         # Интервал логирования метрик (шаги)
    'save_interval': 100000,        # Интервал сохранения чекпоинтов (шаги) 
    'print_interval_episodes': 10, # Как часто выводить прогресс в консоль (эпизоды)
    'cudnn_deterministic': True,   # Для воспроизводимости на CUDA
    'cudnn_benchmark': False,      # Может ускорить, но снизить воспроизводимость

    # --- Параметры MLflow (для TrainingLogger) ---
    'mlflow_tracking_uri': None,  # URI для MLflow сервера 
    'mlflow_experiment_name': 'SAC_Unity', # Название эксперимента
    'mlflow_run_name': None,      # Название запуска, None для автогенерации
    'log_system_info': True,      # Логировать системную информацию
    'log_memory_usage': True,     # Логировать использование памяти

    # --- Параметры обучения (для SACTrainer) ---
    'max_steps': 250000,          # Общее количество шагов обучения
    'episode_max_steps': 25000,   # Максимальное количество шагов в эпизоде
    'batch_size': 128,            # Размер батча для обновления агента
    'update_after': 1000,        # Шаг, после которого начинаются обновления
    'start_steps': 1000,         # Шаги со случайными действиями 
    'updates_per_step': 1,        # Сколько раз вызывать agent.perform_updates за шаг тренера
    'reward_scale': 1.0,          # Масштабирование награды перед добавлением в буфер
    'clear_cuda_cache_interval_episodes': 4, # Как часто очищать кэш CUDA (эпизоды)

    # --- Конфигурация агента (для SACAgentFactory) ---
    'agent_config': {
        'obs_dim': None, 
        'action_dim': None, 
        'device': 'cuda', 

        # -- Архитектура сетей --
        'hidden_dim': 64,
        'num_layers': 2,
        'activation_fn': 'leaky_relu',
        'use_lstm': False,
        'use_layer_norm': True,
        'dropout': 0.0, 

        # -- Параметры оптимизации --
        'optimizer_type': 'adam', 
        'actor_lr': 3e-4,   
        'critic_lr': 3e-4,  
        'alpha_lr': 3e-4,   
        'weight_decay': 0.0,
        'optimizer_kwargs': {}, # Дополнительные аргументы для оптимизатора (напр. betas для Adam)

        # -- Гиперпараметры SAC --
        'gamma': 0.99,
        'tau': 0.005,
        'alpha': 0.2,
        'learn_alpha': True,
        'target_entropy': None, # None = auto-calculate
        'clip_grad_norm_actor': 1.0, 
        'clip_grad_norm_critic': 1.0,
        'clip_grad_norm_alpha': 1.0,


        # -- Конфигурация буфера (для ReplayBufferFactory) --
        'buffer_config': {
            'capacity': 1000000, 
            'use_prioritized': True, # Использовать приоритетный буфер
            'alpha': 0.6,      
            'beta': 0.4,       
            'beta_annealing_steps': 90000, # Формула: (train_steps - update_after) / updates_per_step
            'epsilon': 1e-6,   
            'use_sequence': False,
            'sequence_length': 10,
            'storage_dtype': 'float16' # Тип для хранения данных в буфере
        },

        # -- Конфигурация нормализатора (для NormalizerFactory) --
        'normalizer_config': {
            'use_normalizer': True,
            'normalizer_type': 'welford',
            'clip_range': 10.0,
            'normalizer_epsilon': 1e-8 
        },

        # -- Конфигурация Warm Start (для WarmStartFactory) --
        'warm_start_config': {
            'use_warm_start': False,
            'warm_start_type': 'decision_tree', 
            'warm_start_steps': 10000,
            'warm_start_random_steps': 1000,
            'warm_start_max_depth': 5,
            'warm_start_min_samples_split': 2,
            'warm_start_random_state': 42,
            'warm_start_noise': 0.1 
        },

        'storage_dtype': 'float16' # Общий тип для хранения данных 
    },

    # --- Параметры экспорта ONNX (для SACTrainer) ---
    'export_onnx': True,
    'export_dir': 'onnx_export', # Имя директории для ONNX файлов внутри save_dir
    'onnx_input_shape': None, # Форма входа для ONNX, None для автоопределения (1,1,obs_dim) для LSTM

    # --- Конфигурация окружения (для EnvFactory) ---
    'env_config': {
        # Укажите путь к вашему Unity-окружению
        'env_name': r'C:\Users\artem\Projects\SAC\envs\Windows_Alone\UnityEnvironment.exe',  # Измените на ваш путь
        'file_name': r'C:\Users\artem\Projects\SAC\envs\Windows_Alone\UnityEnvironment.exe',  # Измените на ваш путь
        'worker_id': 1,
        'base_port': 5005, 
        'seed': 42, # Seed для окружения
        'side_channels': [], # Пустой список, если нет SideChannels
        'timeout_wait': 60, # Таймаут ожидания
        'no_graphics': True, # Если не нужна графика Unity
        'time_scale': 20.0,
        'flatten_obs': True,
        'normalize_actions': False,
    }
}

# --- Расчет annealing steps для beta ---
# Общее число шагов обновлений = (train_steps - update_after) / updates_per_step
total_updates = (config['max_steps'] - config['update_after']) // config['updates_per_step']
if total_updates > 0:
    config['agent_config']['buffer_config']['beta_annealing_steps'] = total_updates
else:
    config['agent_config']['buffer_config']['beta_annealing_steps'] = None # Не будет отжига, если обновления не начнутся


### Опция загрузки существующей модели

In [ ]:
# Загружаем существующий чекпоинт
config['load_checkpoint_path'] = 'results/sac_agent_best.pth'

print(f"Обучение на устройстве: {config['device']}")
print(f"Среда: {config['env_config']['env_name']}")
print(f"Количество шагов: {config['max_steps']}")
if config['load_checkpoint_path']:
    print(f"Загрузка чекпоинта из: {config['load_checkpoint_path']}")

## 2. Создание среды для проверки

In [ ]:
env_factory = EnvFactory()
env = env_factory.create_gym_env(
    env_name=config['env_config']['env_name'],
    config=config['env_config']
)

In [ ]:
print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

In [ ]:
observation = env.reset()
print(f"Observation shape: {observation.shape}")

In [ ]:
action = env.action_space.sample()
print(f"Action shape: {action.shape}")

In [ ]:
next_observation, reward, terminated, truncated = env.step(action)
print(f"Next observation shape: {next_observation.shape}")
print(f"Reward: {reward}")
print(f"Terminated: {terminated}")
print(f"Truncated: {truncated}")

In [ ]:
env.close()

## 3. Обучение агентa

In [ ]:
print("Начало обучения...")
start_time = time.time()

# Создаем окружение заново (так как закрыли его выше)
env = env_factory.create_gym_env(
    env_name=config['env_config']['env_name'],
    config=config['env_config']
)

# Запуск процесса обучения
results = train_agent(config, env)

elapsed_time = time.time() - start_time
print(f"Обучение завершено за {elapsed_time:.2f} секунд")
print(f"Результаты: {json.dumps(results, indent=2)}")

## 4. Оценка обученного агента

In [ ]:
eval_config = config.copy()
eval_config['mlflow_experiment_name'] = 'SAC_Evaluation'
eval_config['mlflow_run_name'] = f'eval_run_{time.strftime("%Y%m%d_%H%M%S")}'

In [ ]:
# Путь к лучшей модели
model_path = os.path.join(config['save_dir'], f"{config.get('model_name', 'sac_agent')}_best.pth")
eval_config['load_checkpoint_path'] = model_path

print(f"Оценка модели из: {model_path}")

# Создаем новое окружение для оценки
eval_env = env_factory.create_gym_env(
    env_name=config['env_config']['env_name'],
    config=config['env_config']
)

eval_results = evaluate_agent(
    config=eval_config,
    env=eval_env,
    n_episodes=10,
    deterministic=True
)

print(f"Результаты оценки:")
print(f"Средняя награда: {eval_results['mean_return']:.2f} ± {eval_results['std_return']:.2f}")
print(f"Средняя длина эпизода: {eval_results['mean_length']:.2f}")

# Закрываем окружение оценки
eval_env.close()

## 5. Просмотр результатов в MLflow

In [ ]:
# Запустите MLflow UI если хотите просмотреть результаты (в отдельном терминале)
# !mlflow ui

# Или откройте браузер с адресом
print("Откройте браузер по адресу http://localhost:5000 для просмотра результатов MLflow")